## Pitch, Roll, and Yaw (PEO) Extraction and Visualization

This notebook is designed to extract and visualize Pitch, Roll, and Yaw (PEO) data from a flight log (`.ulg` file). 

### What are Pitch, Roll, and Yaw?
- **Pitch**: The up or down tilt of the vehicle's nose (rotation around the lateral axis).
- **Roll**: The side-to-side tilt of the vehicle (rotation around the longitudinal axis).
- **Yaw**: The direction the vehicle is pointing (rotation around the vertical axis).

These angles, collectively known as Euler angles, describe the orientation of a vehicle in 3D space. They are extracted from the flight log data, converted from quaternions for better human interpretability, and plotted to analyze the vehicle's behavior during the flight.

### Purpose
This notebook will:
1. Extract actual orientation data from the `vehicle_attitude` dataset in quaternions and convert it to Euler angles.
2. Extract control setpoints for Pitch, Roll, and Yaw from the `vehicle_rates_setpoint` dataset (if available).
3. Visualize both actual orientation and setpoints to analyze vehicle performance.


In [73]:
import math
import matplotlib.pyplot as plt
from pyulog import ULog

In [74]:
def quaternion_to_euler(q0, q1, q2, q3):
    """
    Convert quaternions to Euler angles (Roll, Pitch, Yaw).
    :param q0, q1, q2, q3: Quaternion values.
    :return: roll, pitch, yaw (in radians).
    """
    roll = math.atan2(2.0 * (q0 * q1 + q2 * q3), 1.0 - 2.0 * (q1**2 + q2**2))
    sinp = 2.0 * (q0 * q2 - q3 * q1)
    pitch = math.asin(sinp) if abs(sinp) <= 1 else math.copysign(math.pi / 2, sinp)
    yaw = math.atan2(2.0 * (q0 * q3 + q1 * q2), 1.0 - 2.0 * (q2**2 + q3**2))
    return roll, pitch, yaw

In [75]:
def extract_peo(ulog):
    # Extract actual PEO from vehicle_attitude
    attitude_data = ulog.get_dataset("vehicle_attitude")
    timestamps_att = attitude_data.data['timestamp']
    q0, q1, q2, q3 = (attitude_data.data['q[0]'], 
                       attitude_data.data['q[1]'], 
                       attitude_data.data['q[2]'], 
                       attitude_data.data['q[3]'])
    
    peo_att = []
    for t, a, b, c, d in zip(timestamps_att, q0, q1, q2, q3):
        roll, pitch, yaw = quaternion_to_euler(a, b, c, d)
        peo_att.append((t, roll, pitch, yaw))
    
    # Extract setpoint PEO from vehicle_rates_setpoint
    try:
        rates_setpoint_data = ulog.get_dataset("vehicle_rates_setpoint")
        timestamps_set = rates_setpoint_data.data['timestamp']
        roll_sp = rates_setpoint_data.data['roll']
        pitch_sp = rates_setpoint_data.data['pitch']
        yaw_sp = rates_setpoint_data.data['yaw']
        peo_set = list(zip(timestamps_set, roll_sp, pitch_sp, yaw_sp))
    except KeyError:
        print("vehicle_rates_setpoint not available in the log.")
        peo_set = []

    return peo_att, peo_set

In [ ]:
def plot_peo(peo_att, peo_set, plot_type='both'):
    """
    Plot Pitch, Roll, and Yaw (PEO) data.

    Parameters:
    - peo_att: List of tuples [(timestamp, roll, pitch, yaw)] for actual PEO values.
    - peo_set: List of tuples [(timestamp, roll_setpoint, pitch_setpoint, yaw_setpoint)] for setpoint PEO values.
    - plot_type: Type of plot to generate:
        - 'actual': Plot actual PEO values only.
        - 'setpoint': Plot setpoint PEO values only.
        - 'both': Plot both actual and setpoint on the same graph.
        - 'separate': Create separate graphs for Roll, Pitch, and Yaw.
        - 'deviation': Highlight differences between actual and setpoint.
    """
    if plot_type in ['actual', 'both', 'setpoint']:
        plt.figure(figsize=(12, 6))

    # Extract data
    if peo_att:
        timestamps, roll, pitch, yaw = zip(*peo_att)
    if peo_set:
        timestamps_sp, roll_sp, pitch_sp, yaw_sp = zip(*peo_set)

    # 1. Plot Actual Only
    if plot_type == 'actual':
        plt.plot(timestamps, roll, label='Roll (actual)', color='blue')
        plt.plot(timestamps, pitch, label='Pitch (actual)', color='green')
        plt.plot(timestamps, yaw, label='Yaw (actual)', color='red')
        plt.title('Pitch, Roll, Yaw - Actual Only')

    # 2. Plot Setpoint Only
    elif plot_type == 'setpoint':
        plt.plot(timestamps_sp, roll_sp, '--', label='Roll (setpoint)', color='cyan')
        plt.plot(timestamps_sp, pitch_sp, '--', label='Pitch (setpoint)', color='lime')
        plt.plot(timestamps_sp, yaw_sp, '--', label='Yaw (setpoint)', color='magenta')
        plt.title('Pitch, Roll, Yaw - Setpoint Only')

    # 3. Plot Both
    elif plot_type == 'both':
        plt.plot(timestamps, roll, label='Roll (actual)', color='blue')
        plt.plot(timestamps, pitch, label='Pitch (actual)', color='green')
        plt.plot(timestamps, yaw, label='Yaw (actual)', color='red')
        plt.plot(timestamps_sp, roll_sp, '--', label='Roll (setpoint)', color='cyan')
        plt.plot(timestamps_sp, pitch_sp, '--', label='Pitch (setpoint)', color='lime')
        plt.plot(timestamps_sp, yaw_sp, '--', label='Yaw (setpoint)', color='magenta')
        plt.title('Pitch, Roll, Yaw - Actual vs Setpoint')

    # 4. Separate Graphs for Each Axis
    elif plot_type == 'separate':
        fig, axs = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
        
        # Roll
        axs[0].plot(timestamps, roll, label='Roll (actual)', color='blue')
        axs[0].plot(timestamps_sp, roll_sp, '--', label='Roll (setpoint)', color='cyan')
        axs[0].set_title('Roll - Actual vs Setpoint')
        axs[0].set_ylabel('Angle [rad]')
        axs[0].legend()
        axs[0].grid()
        
        # Pitch
        axs[1].plot(timestamps, pitch, label='Pitch (actual)', color='green')
        axs[1].plot(timestamps_sp, pitch_sp, '--', label='Pitch (setpoint)', color='lime')
        axs[1].set_title('Pitch - Actual vs Setpoint')
        axs[1].set_ylabel('Angle [rad]')
        axs[1].legend()
        axs[1].grid()
        
        # Yaw
        axs[2].plot(timestamps, yaw, label='Yaw (actual)', color='red')
        axs[2].plot(timestamps_sp, yaw_sp, '--', label='Yaw (setpoint)', color='magenta')
        axs[2].set_title('Yaw - Actual vs Setpoint')
        axs[2].set_xlabel('Time [ms]')
        axs[2].set_ylabel('Angle [rad]')
        axs[2].legend()
        axs[2].grid()
        
        plt.tight_layout()
        plt.show()
        return

    # 5. Highlight Differences Between Actual and Setpoint
    elif plot_type == 'deviation':
        fig, axs = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
        
        # Roll
        axs[0].plot(timestamps, roll, label='Roll (actual)', color='blue')
        axs[0].plot(timestamps_sp, roll_sp, '--', label='Roll (setpoint)', color='cyan')
        axs[0].fill_between(timestamps, roll, roll_sp, color='blue', alpha=0.1, label='Deviation')
        axs[0].set_title('Roll - Deviation Highlight')
        axs[0].set_ylabel('Angle [rad]')
        axs[0].legend()
        axs[0].grid()
        
        # Pitch
        axs[1].plot(timestamps, pitch, label='Pitch (actual)', color='green')
        axs[1].plot(timestamps_sp, pitch_sp, '--', label='Pitch (setpoint)', color='lime')
        axs[1].fill_between(timestamps, pitch, pitch_sp, color='green', alpha=0.1, label='Deviation')
        axs[1].set_title('Pitch - Deviation Highlight')
        axs[1].set_ylabel('Angle [rad]')
        axs[1].legend()
        axs[1].grid()
        
        # Yaw
        axs[2].plot(timestamps, yaw, label='Yaw (actual)', color='red')
        axs[2].plot(timestamps_sp, yaw_sp, '--', label='Yaw (setpoint)', color='magenta')
        axs[2].fill_between(timestamps, yaw, yaw_sp, color='red', alpha=0.1, label='Deviation')
        axs[2].set_title('Yaw - Deviation Highlight')
        axs[2].set_xlabel('Time [ms]')
        axs[2].set_ylabel('Angle [rad]')
        axs[2].legend()
        axs[2].grid()
        
        plt.tight_layout()
        plt.show()
        return

    # Display the graph for the first three cases
    plt.xlabel('Time [ms]')
    plt.ylabel('Angle [rad]')
    plt.legend()
    plt.grid()
    plt.show()


In [77]:
def main():
    # Replace with your ULog file path
    ulog_file = "/home/hawk/GitDir/Logging/2024-12-09/12_40_59.ulg"
    ulog = ULog(ulog_file)
    
    peo_att = extract_peo_actual(ulog)
    peo_set = extract_peo_setpoint(ulog)

    plot_peo(peo_att, peo_set, plot_type='actual')        # Only actual data
    plot_peo(peo_att, peo_set, plot_type='setpoint')      # Only setpoint data
    plot_peo(peo_att, peo_set, plot_type='both')          # Both actual and setpoint
    plot_peo(peo_att, peo_set, plot_type='separate')      # Separate graphs for Roll, Pitch, Yaw
    plot_peo(peo_att, peo_set, plot_type='deviation')     # Highlight differences



In [78]:
if __name__ == "__main__":
    main()

Error extracting PEO (actual): 'q'
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


UnboundLocalError: local variable 'timestamps' referenced before assignment

<Figure size 1200x600 with 0 Axes>